In [ ]:
import pygame
import sys
import random

# --- Initialize pygame ---
pygame.init()

# --- Window setup ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("A Dog's Way Bone")

# --- Colors ---
GREEN = (34, 139, 34)
WHITE = (255, 255, 255)

# --- Game setup ---
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

# --- Load PNG images (replace with your file paths) ---
dog_img = pygame.image.load("dog.png").convert_alpha()
doghouse_img = pygame.image.load("doghouse.png").convert_alpha()
tree_img = pygame.image.load("tree.png").convert_alpha()
bone_img = pygame.image.load("bone.png").convert_alpha()

# --- Scale images ---
dog_img = pygame.transform.scale(dog_img, (40, 40))
doghouse_img = pygame.transform.scale(doghouse_img, (60, 60))
tree_img = pygame.transform.scale(tree_img, (50, 70))
bone_img = pygame.transform.scale(bone_img, (25, 25))


def generate_non_overlapping_objects(num_objects, size_func, existing_rects=None):
    """Generate non-overlapping rectangles given size and existing rects."""
    objects = []
    existing_rects = existing_rects or []

    while len(objects) < num_objects:
        width, height = size_func()
        rect = pygame.Rect(
            random.randint(0, WIDTH - width),
            random.randint(0, HEIGHT - height),
            width,
            height
        )

        # Check overlap with all existing rects
        if any(rect.colliderect(other) for other in existing_rects + objects):
            continue  # Try another position

        objects.append(rect)
    return objects


def start_game():
    """Set up a new game state."""
    dog = pygame.Rect(100, 100, 40, 40)
    doghouse = pygame.Rect(WIDTH - 100, HEIGHT - 100, 60, 60)

    trees = generate_non_overlapping_objects(
        5,
        lambda: (50, 70)
    )

    bones = generate_non_overlapping_objects(
        3,
        lambda: (25, 25),
        existing_rects=trees
    )

    return dog, doghouse, trees, bones, 0, False


# --- Initialize first game state ---
dog, doghouse, trees, bones, bones_collected, game_over = start_game()

# --- Main Game Loop ---
running = True
speed = 5

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        # Restart the game
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                dog, doghouse, trees, bones, bones_collected, game_over = start_game()
            if event.key == pygame.K_ESCAPE:
                running = False

    keys = pygame.key.get_pressed()

    if not game_over:
        # Movement
        if keys[pygame.K_LEFT]:
            dog.x -= speed
        if keys[pygame.K_RIGHT]:
            dog.x += speed
        if keys[pygame.K_UP]:
            dog.y -= speed
        if keys[pygame.K_DOWN]:
            dog.y += speed

        # Keep dog inside the screen
        dog.x = max(0, min(WIDTH - dog.width, dog.x))
        dog.y = max(0, min(HEIGHT - dog.height, dog.y))

        # Collision with trees
        for tree in trees:
            if dog.colliderect(tree):
                if keys[pygame.K_LEFT]:
                    dog.x += speed
                if keys[pygame.K_RIGHT]:
                    dog.x -= speed
                if keys[pygame.K_UP]:
                    dog.y += speed
                if keys[pygame.K_DOWN]:
                    dog.y -= speed

        # Collect bones
        for bone in bones[:]:
            if dog.colliderect(bone):
                bones.remove(bone)
                bones_collected += 1

        # Win condition
        if bones_collected == 3 and dog.colliderect(doghouse):
            game_over = True

    # --- Drawing ---
    screen.fill(GREEN)
    screen.blit(doghouse_img, doghouse)
    for tree in trees:
        screen.blit(tree_img, tree)
    for bone in bones:
        screen.blit(bone_img, bone)
    screen.blit(dog_img, dog)

    # --- Text ---
    text = font.render(f"Bones Collected: {bones_collected}/3", True, WHITE)
    screen.blit(text, (10, 10))

    if game_over:
        win_text = font.render("You Win! 🦴🐕 Press [R] to Restart", True, WHITE)
        screen.blit(win_text, (WIDTH // 2 - 200, HEIGHT // 2 - 20))

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 